Import packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Importing Dataset

In [2]:
df_train = pd.read_csv("train.csv")#.dropna(axis=0, subset=["SalePrice"], inplace=True)
df_test = pd.read_csv("test.csv")

In [3]:
# x = df_train.iloc[:, [1, 4, 7, 10, 12, 16, 17, 18, 19, 20, 21, 38, 46, 49, 50, 54, 62]]
x = df_train.iloc[:, :-1]
y = df_train.iloc[:, -1]
x_values = x.values
y_values = y.values

In [4]:
cols_with_missing = [col for col in df_train.columns 
                                 if df_train[col].isnull().any()]                                  
candidate_train_predictors = df_train.drop(['Id', 'SalePrice'] + cols_with_missing, axis=1)
candidate_test_predictors = df_test.drop(['Id'] + cols_with_missing, axis=1)

In [5]:
low_cardinality_cols = [cname for cname in candidate_train_predictors.columns if 
                                candidate_train_predictors[cname].nunique() < 10 and
                                candidate_train_predictors[cname].dtype == "object"]
numeric_cols = [cname for cname in candidate_train_predictors.columns if 
                                candidate_train_predictors[cname].dtype in ['int64', 'float64']]
my_cols = low_cardinality_cols + numeric_cols
train_predictors = candidate_train_predictors[my_cols]
test_predictors = candidate_test_predictors[my_cols]

In [6]:
train_predictors.dtypes.sample(10)

Condition2     object
ScreenPorch     int64
FullBath        int64
OverallCond     int64
MiscVal         int64
3SsnPorch       int64
LotConfig      object
CentralAir     object
ExterQual      object
LotArea         int64
dtype: object

Object = cateogrical. We need to onehot encode these columns

In [7]:
one_hot_encoded_training_predictors = pd.get_dummies(train_predictors)
one_hot_encoded_training_predictors 

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,8450,7,5,2003,2003,706,0,150,856,...,0,0,0,1,0,0,0,0,1,0
1,20,9600,6,8,1976,1976,978,0,284,1262,...,0,0,0,1,0,0,0,0,1,0
2,60,11250,7,5,2001,2002,486,0,434,920,...,0,0,0,1,0,0,0,0,1,0
3,70,9550,7,5,1915,1970,216,0,540,756,...,0,0,0,1,1,0,0,0,0,0
4,60,14260,8,5,2000,2000,655,0,490,1145,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,7917,6,5,1999,2000,0,0,953,953,...,0,0,0,1,0,0,0,0,1,0
1456,20,13175,6,6,1978,1988,790,163,589,1542,...,0,0,0,1,0,0,0,0,1,0
1457,70,9042,7,9,1941,2006,275,0,877,1152,...,0,0,0,1,0,0,0,0,1,0
1458,20,9717,5,6,1950,1996,49,1029,0,1078,...,0,0,0,1,0,0,0,0,1,0


In [8]:
one_hot_encoded_training_predictors = pd.get_dummies(train_predictors)
one_hot_encoded_test_predictors = pd.get_dummies(test_predictors)
final_train, final_test = one_hot_encoded_training_predictors.align(one_hot_encoded_test_predictors,
                                                                    join='left', 
                                                                    axis=1)

In [9]:
final_test

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,20,11622,5,6,1961,1961,468.0,144.0,270.0,882.0,...,0,0,0,1,0,0,0,0,1,0
1,20,14267,6,6,1958,1958,923.0,0.0,406.0,1329.0,...,0,0,0,1,0,0,0,0,1,0
2,60,13830,5,5,1997,1998,791.0,0.0,137.0,928.0,...,0,0,0,1,0,0,0,0,1,0
3,60,9978,6,6,1998,1998,602.0,0.0,324.0,926.0,...,0,0,0,1,0,0,0,0,1,0
4,120,5005,8,5,1992,1992,263.0,0.0,1017.0,1280.0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,1936,4,7,1970,1970,0.0,0.0,546.0,546.0,...,0,0,0,1,0,0,0,0,1,0
1455,160,1894,4,5,1970,1970,252.0,0.0,294.0,546.0,...,0,0,0,1,1,0,0,0,0,0
1456,20,20000,5,7,1960,1996,1224.0,0.0,0.0,1224.0,...,0,0,0,1,1,0,0,0,0,0
1457,85,10441,5,5,1992,1992,337.0,0.0,575.0,912.0,...,0,0,0,1,0,0,0,0,1,0


In [10]:
y

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [11]:
print(final_train.shape)

(1460, 159)


In [12]:
print(y.shape)

(1460,)


In [13]:
from sklearn.model_selection import train_test_split 
(x_train, x_test, y_train, y_test) = train_test_split(final_train.values, y, test_size=1/3, random_state=0) 

In [14]:
x_train.shape

(973, 159)

In [15]:
y_train.shape

(973,)

Multiple Linear Regression

In [16]:
from sklearn.linear_model import LinearRegression
l = LinearRegression()
l.fit(x_train, y_train)
y_pred = l.predict(x_test)
from sklearn import metrics
# Evaluating the model and printing the results using MAE
print(metrics.mean_absolute_error(y_test, y_pred))

# Evaluating the model and printing the results using MSE
print(metrics.mean_squared_error(y_test, y_pred))

# Importing the math library
import math

# Evaluating the model and printing the results using RMSE
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# Importing metrics library
from sklearn import metrics
# Evaluating the model and printing the results using MAE
final = metrics.mean_absolute_error(y_test, y_pred)

# Evaluating the model and printing the results using MSE
print(metrics.mean_squared_error(y_test, y_pred))

22815.5947942632
2443972227.366738
49436.547486315605
2443972227.366738


In [19]:
#submission = pd.concat([df_test["Id"], pd.Series(final, name='SalePrice')], axis=1)
submission = pd.concat([df_test["Id"], pd.Series(y, name='SalePrice')], axis=1)
submission.to_csv('./submission.csv', index=False, header=True)
submission 

,Id,SalePrice
0,1461.0,208500
1,1462.0,181500
2,1463.0,223500
3,1464.0,140000
4,1465.0,250000
...,...,...
1455,2916.0,175000
1456,2917.0,210000
1457,2918.0,266500
1458,2919.0,142125


In [20]:
final_test.iloc[:, -1]

0       0
1       0
2       0
3       0
4       0
       ..
1454    0
1455    0
1456    0
1457    0
1458    0
Name: SaleCondition_Partial, Length: 1459, dtype: uint8

In [21]:
#submission = pd.concat([df_test["Id"], pd.Series(final, name='SalePrice')], axis=1)
submission = pd.concat([df_test["Id"], pd.Series(y, name='SalePrice')], axis=1)
submission.to_csv('./submission.csv', index=False, header=True)
submission 

,Id,SalePrice
0,1461.0,208500
1,1462.0,181500
2,1463.0,223500
3,1464.0,140000
4,1465.0,250000
...,...,...
1455,2916.0,175000
1456,2917.0,210000
1457,2918.0,266500
1458,2919.0,142125


KNN

In [30]:
from sklearn.preprocessing import StandardScaler
scale_X = StandardScaler()
X_train = scale_X.fit_transform(x_train)
X_test = scale_X.transform(x_test)

In [32]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric =  'minkowski', p = 2)
classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [38]:
y_pred2 = classifier.predict(X_test)

In [39]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred2)

In [40]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred2)
print('Accuracy is %f'% accuracy)

Accuracy is 0.006160


In [43]:
#submission = pd.concat([df_test["Id"], pd.Series(final, name='SalePrice')], axis=1)
submission = pd.concat([df_test["Id"], pd.Series(y_pred2, name='SalePrice')], axis=1)
submission.to_csv('./submission.csv', index=False, header=True)
submission 

,Id,SalePrice
0,1461,127000.0
1,1462,97500.0
2,1463,127000.0
3,1464,140000.0
4,1465,83000.0
...,...,...
1454,2915,NaN
1455,2916,NaN
1456,2917,NaN
1457,2918,NaN
